validation set. importa il db di sklearn

In [2]:
from sklearn.datasets import load_diabetes
from sklearn import svm
from sklearn.model_selection import train_test_split, GridSearchCV
import pandas as pd

In [3]:
diabetes = load_diabetes()

In [4]:
diabetes.target[:3]

array([151.,  75., 141.])

In [5]:
diabetes.DESCR

'.. _diabetes_dataset:\n\nDiabetes dataset\n----------------\n\nTen baseline variables, age, sex, body mass index, average blood\npressure, and six blood serum measurements were obtained for each of n =\n442 diabetes patients, as well as the response of interest, a\nquantitative measure of disease progression one year after baseline.\n\n**Data Set Characteristics:**\n\n  :Number of Instances: 442\n\n  :Number of Attributes: First 10 columns are numeric predictive values\n\n  :Target: Column 11 is a quantitative measure of disease progression one year after baseline\n\n  :Attribute Information:\n      - age     age in years\n      - sex\n      - bmi     body mass index\n      - bp      average blood pressure\n      - s1      tc, total serum cholesterol\n      - s2      ldl, low-density lipoproteins\n      - s3      hdl, high-density lipoproteins\n      - s4      tch, total cholesterol / HDL\n      - s5      ltg, possibly log of serum triglycerides level\n      - s6      glu, blood sugar

In [6]:
diabetes.data[:5]

array([[ 0.03807591,  0.05068012,  0.06169621,  0.02187239, -0.0442235 ,
        -0.03482076, -0.04340085, -0.00259226,  0.01990749, -0.01764613],
       [-0.00188202, -0.04464164, -0.05147406, -0.02632753, -0.00844872,
        -0.01916334,  0.07441156, -0.03949338, -0.06833155, -0.09220405],
       [ 0.08529891,  0.05068012,  0.04445121, -0.00567042, -0.04559945,
        -0.03419447, -0.03235593, -0.00259226,  0.00286131, -0.02593034],
       [-0.08906294, -0.04464164, -0.01159501, -0.03665608,  0.01219057,
         0.02499059, -0.03603757,  0.03430886,  0.02268774, -0.00936191],
       [ 0.00538306, -0.04464164, -0.03638469,  0.02187239,  0.00393485,
         0.01559614,  0.00814208, -0.00259226, -0.03198764, -0.04664087]])

In [7]:
diabetes_df = pd.DataFrame(data=diabetes.data, columns=diabetes.feature_names)
diabetes_df['target'] = diabetes.target
diabetes_df

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6,target
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019907,-0.017646,151.0
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068332,-0.092204,75.0
2,0.085299,0.050680,0.044451,-0.005670,-0.045599,-0.034194,-0.032356,-0.002592,0.002861,-0.025930,141.0
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022688,-0.009362,206.0
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031988,-0.046641,135.0
...,...,...,...,...,...,...,...,...,...,...,...
437,0.041708,0.050680,0.019662,0.059744,-0.005697,-0.002566,-0.028674,-0.002592,0.031193,0.007207,178.0
438,-0.005515,0.050680,-0.015906,-0.067642,0.049341,0.079165,-0.028674,0.034309,-0.018114,0.044485,104.0
439,0.041708,0.050680,-0.015906,0.017293,-0.037344,-0.013840,-0.024993,-0.011080,-0.046883,0.015491,132.0
440,-0.045472,-0.044642,0.039062,0.001215,0.016318,0.015283,-0.028674,0.026560,0.044529,-0.025930,220.0


**PROBABILMENTE I DATI SONO STATI NORMALIZZATI, E' UN BENE PERCHE' NON ABBIAMO PROBLEMI DI POSSIBILI OUTLIERS

SCRIVI PER PIACERE CHE I DATI SONO STATI NORMALIZZATI INSERENDO L'ESEMPIO DEL DB
**

creo i due array per poterli inserire nella grid search

In [8]:
X, y = diabetes.data, diabetes.target
print(diabetes.feature_names)

['age', 'sex', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's5', 's6']


divido il dataset in training set e test set

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

carico il modello di apprendimento che per il momento utilizzo l'*SVC*

In [10]:
svr = svm.SVR()

inserisco i possibili parametri che gridsearchcv combinerà

In [11]:
parameters = {
    'kernel': ('linear', 'rbf', 'poly'),
    'gamma': [0.001, 0.01, 0.1, 0.5, 0.8, 1, 3],
    'degree': [2,3],
    'C': [0.001, 0.01, 0.1, 0.5, 0.8, 1, 3, 5, 10, 20, 50, 100, 200, 500, 1000]
}

dopo creato i parametri, posso darli in pasto a *GridsearchCV*

In [12]:
clf = GridSearchCV(svr, parameters, cv=5)

faccio il training di dati con la gridsearchcv

In [17]:
clf.fit(X_train, y_train)
clf.best_estimator_, clf.best_score_

(SVR(C=100, degree=2, gamma=3), 0.4808977400025807)

ho ottenuto questi parametri:
1.   C=100
2.   degree=2
3.   gamma=3

adesso vedrò la precisione con i dati di training e poi con quelli di test


In [20]:
new_svr = svm.SVR(C=100, gamma=3.0, degree=2)
new_svr.fit(X_train, y_train)
new_svr.score(X_test, y_test)

0.5090204897985313